## Scraping antara news

In [17]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading

In [18]:
jumlah_index = 1
threads_link = []
links = []
results = []
threads = []

In [19]:
def scrape_links(page_number):
    url = f"https://www.antaranews.com/politik/{page_number}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('article',{"class": "simple-post simple-big clearfix"})
    
    page_links = []
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)
    
    print(f"Scraped {len(page_links)} links from page {page_number}")
    return page_links

In [20]:
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))

Scraped 15 links from page 1
Total Links: 15


In [21]:
def scrape_url(url):
    try:
        headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
        }
        response = requests.get(url,headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')        
            # Judul Berita
            title_elem = soup.find('h1',{"class": "post-title"})
            if title_elem:
                title_text = title_elem.text
            else:
                title_text = "Title not found"
            # Author berita
            # author_elem = soup.find('div', {"class": "tags-wrapper"})
            # if author_elem:
            #     author_text = author_elem.find('span')
            #     author_text = author_text.text.split(':')[1]
            # else:
            author_text = "Author not found"     
            # tanggal berita
            date_elem = soup.find('span', {"class": "article-date"})
            if date_elem:
                date_text = date_elem.text.strip()
            else:
                date_text = "Date not found"
            #     # Category berita
            # category_elements = soup.find('ul', {"class": "breadcrumb"})
            # if category_elements:
            #     category_text = category_elements.find_all('li')
            #     category_text= category_text[2].get_text()
            # else:
            category_text = "Category not found"
            #     # Content Berita
            body_elem = soup.find('div', {"class": "post-content clearfix"})
            if body_elem:
                 content_text = body_elem.text
            else:
                content_text ="Content not found"

            results.append({'title': title_text,
                            'author' : author_text,
                            'category':category_text,
                            'date': date_text,
                            'content' : content_text,
                            'link' : url})
        else:
            print(f"Failed to retrieve data from {url}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL '{url}': {e}")
    except Exception as e:
        print(f"Error processing URL '{url}': {e}")

In [22]:
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

In [23]:
df = pd.DataFrame(results)
print("hasil data scrapping",len(results))
df.head(100)

15


,title,author,category,date,content,region,link
0,Politikus PKB harap Said Aqil Siroj jadi Ketua...,Author not found,Category not found,"Kamis, 28 September 2023 23:07 WIB",\n\t\t\t\tJakarta (ANTARA) - Ketua Bidang Huku...,banten,https://www.antaranews.com/berita/3748290/poli...
1,Kasad menilai meriam-meriam TNI AD perlu moder...,Author not found,Category not found,"Jumat, 29 September 2023 00:45 WIB","\nSaya merasa bangga, prajurit-prajurit Arhanu...",banten,https://www.antaranews.com/berita/3748344/kasa...
2,Prabowo akui Luhut salah satu jenderal terbaik...,Author not found,Category not found,"Kamis, 28 September 2023 23:12 WIB",\n\t\t\t\tJakarta (ANTARA) - Menteri Pertahana...,banten,https://www.antaranews.com/berita/3748296/prab...
3,Muhaimin optimistis pasangan AMIN bisa menang ...,Author not found,Category not found,"Kamis, 28 September 2023 22:57 WIB","\n\t\t\t\tJember, Jawa Timur (ANTARA) - Ketua ...",banten,https://www.antaranews.com/berita/3748281/muha...
4,Anies-Muhaimin kobarkan spirit perubahan saat ...,Author not found,Category not found,"Kamis, 28 September 2023 20:49 WIB","\n""PKB setuju dengan Koalisi Perubahan. Saya s...",banten,https://www.antaranews.com/berita/3748176/anie...
5,PPP harap teruskan nostalgia seperti di 2019 d...,Author not found,Category not found,"Kamis, 28 September 2023 20:33 WIB",\n\t\t\t\tJakarta (ANTARA) - Ketua DPP Partai ...,banten,https://www.antaranews.com/berita/3748152/ppp-...
6,SMRC: Ganjar-Mahfud unggul dengan 45 persen di...,Author not found,Category not found,"Kamis, 28 September 2023 21:16 WIB",\n\t\t\t\tJakarta (ANTARA) - Saiful Mujani Res...,banten,https://www.antaranews.com/berita/3748209/smrc...
7,Ganjar memperingati maulid nabi bersama majeli...,Author not found,Category not found,"Jumat, 29 September 2023 00:50 WIB",\nIni merupakan salah satu kegiatan yang dilak...,banten,https://www.antaranews.com/berita/3748347/ganj...
8,Survei LSI Denny JA: Elektabilitas Prabowo ung...,Author not found,Category not found,"Kamis, 28 September 2023 23:31 WIB",\n\t\t\t\tPurwakarta (ANTARA) - Hasil survei t...,banten,https://www.antaranews.com/berita/3748311/surv...
9,Hasto Kristiyanto tinjau gladi kotor Rakernas ...,Author not found,Category not found,"Kamis, 28 September 2023 20:03 WIB",\n\t\t\t\tJakarta (ANTARA) - Sekretaris Jender...,banten,https://www.antaranews.com/berita/3748113/hast...


Tempat Simpan Save

In [24]:
# current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
# excel_file_name = f'../tempat_simpan_nasional/antaranews_{current_datetime}.xlsx'
# df.to_excel(excel_file_name, index=False)

# print(f'Data has been saved to {excel_file_name}')

Data has been saved to ../tempat_simpan_nasional/antara_2023-09-29_08-08-15.xlsx
